In [3]:
from pyoculus.problems import AnalyticCylindricalBfield
from pyoculus.solvers import PoincarePlot, FixedPoint, Manifold
%matplotlib ipympl
import matplotlib.pyplot as plt
plt.style.use('lateky')

import numpy as np
import pickle

In [4]:
# Creating the pyoculus problem object, adding the perturbation here use the R, Z provided as center point
pyoproblem = AnalyticCylindricalBfield(
    6,
    0,
    0.8875,
    0.2
)

In [5]:
# set up the integrator for the Poincare
iparams = dict()
iparams["rtol"] = 1e-10

# set up the Poincare plot
pparams = dict()
pparams["nPtrj"] = 10
pparams["nPpts"] = 300
pparams["zeta"] = 0

# Set RZs for the normal (R-only) computation
pparams["Rbegin"] = pyoproblem._R0+1e-3
pparams["Rend"] = 8

# Set up the Poincare plot object
pplot = PoincarePlot(pyoproblem, pparams, integrator_params=iparams)

# # R-only computation
pplot.compute()

In [ ]:
plt.close()
fig, ax = pplot.plot(marker=".", s=1)
# # plt.show()
# # fig.canvas.toolbar_visible = True
# fig.canvas.toolbar_visible = 'fade-in-fade-out'
# # fig.canvas.toolbar_position = 'top'
# fig.canvas.header_visible = False
# fig.canvas.footer_visible = True
# fig.canvas.resizable = False
# # fig.canvas.capture_scroll = True

# # fig.canvas.toolbar_visible = True
# display(fig.canvas)
fig

In [ ]:
pplot.compute_iota()

In [ ]:
pplot.plot_q()
plt.show()

#### Adding the perturbation

In [ ]:
maxwellboltzmann = {"m": 3, "n": -2, "d": 1/np.sqrt(2), "type": "maxwell-boltzmann", "amplitude": 1e-4}
pyoproblem.add_perturbation(maxwellboltzmann)

In [ ]:
# set up the integrator for the Poincare
iparams = dict()
iparams["rtol"] = 1e-10

# set up the Poincare plot
pparams = dict()
pparams["nPtrj"] = 15
pparams["nPpts"] = 150
pparams["zeta"] = 0

# Set RZs for the normal (R-only) computation
pparams["Rbegin"] = pyoproblem._R0+1e-3
pparams["Rend"] = 7.5

# Set up the Poincare plot object
pplot_perturbed = PoincarePlot(pyoproblem, pparams, integrator_params=iparams)

# # R-only computation
pplot_perturbed.compute()

In [ ]:
plt.close()

In [ ]:
pplot_perturbed.plot(marker=".", s=1)
fig_perturbed = plt.gcf()
ax_perturbed = fig_perturbed.gca()

In [ ]:
pyoproblem.plot_intensities([4, 8], [-2, 2], ax=ax_perturbed, alpha=0.3)
display(fig_perturbed.canvas)

In [ ]:
guess = [5, 0.]
ax_perturbed.scatter(*guess)
fig_perturbed

In [ ]:
# set up the integrator for the FixedPoint
iparams = dict()
iparams["rtol"] = 1e-13

pparams = dict()
pparams["nrestart"] = 0
pparams["niter"] = 300
pparams['Z'] = 0

# set up the FixedPoint object
fixedpoint = FixedPoint(pyoproblem, pparams, integrator_params=iparams)

# find the X-point
fixedpoint.compute(guess=guess, pp=2, qq=3, sbegin=2, send=10, tol=1e-10)

if fixedpoint.successful:
    results = [list(p) for p in zip(fixedpoint.x, fixedpoint.y, fixedpoint.z)]
else:
    raise ValueError("X-point not found")

In [ ]:
results
for rr in results:
    ax_perturbed.scatter(rr[0], rr[2], marker="X", edgecolors="black", linewidths=1)
fig_perturbed

In [ ]:
# Set up the manifold
iparams = dict()
iparams["rtol"] = 1e-13
manifold = Manifold(fixedpoint, pyoproblem, integrator_params=iparams)

In [ ]:
manifold.choose(0,1, directions=["u+", "s+"], sngs=[-1, 1])

In [ ]:
manifold.compute(neps=100, nintersect=8)

In [ ]:
r1 = manifold.integrate_single(manifold.rfp_u + 0.1 * manifold.vector_u, 56, 1, ret_jacobian=False)
r2 = manifold.integrate_single(manifold.rfp_s + 0.1 * manifold.vector_s, 57, -1, ret_jacobian=False)

In [ ]:
ax_perturbed.scatter(*r1, marker="o", edgecolors="black", linewidths=1)
ax_perturbed.scatter(*r2, marker="s", edgecolors="black", linewidths=1)

In [ ]:
manifold.plot(ax=ax_perturbed)
fig_perturbed

In [ ]:
manifold.vector_s, manifold.vector_u

In [ ]:
print(manifold.error_linear_regime(0.001, manifold.rfp_u, manifold.vector_u))
print(manifold.error_linear_regime(0.001, manifold.rfp_s, manifold.vector_s, direction=-1))

In [ ]:
manifold.find_N(1e-3, 1e-3)

In [ ]:
manifold.find_homoclinic(1e-3, 1e-3)

In [ ]:
manifold.find_clinics(n_points=4)

In [ ]:
manifold.clinics

In [ ]:
marker = ["X", "o", "s", "p", "P", "*", "x", "D", "d", "^", "v", "<", ">"]
for i, clinic in enumerate(manifold.clinics):
    eps_s_i, eps_u_i = clinic[1:3]

    n_u = 8
    
    # hs_i = manifold.integrate(manifold.rfp_s + eps_s_i * manifold.vector_s, n_s, -1)
    hu_i = manifold.integrate(manifold.rfp_u + eps_u_i * manifold.vector_u, n_u, 1)
    # ax.scatter(hs_i[0,:], hs_i[1,:], marker=marker[i], color="royalblue", edgecolor='cyan', zorder=10)
    ax_perturbed.scatter(hu_i[0,:], hu_i[1,:], marker=marker[i], color="royalblue", edgecolor='cyan', zorder=10, label=f'$h_{i+1}$')

In [ ]:
fig_perturbed.set_size_inches(12, 12)
ax_perturbed.set_xlim(4.4, 6.8)
ax_perturbed.set_ylim(-0.3, 1.4)
ax_perturbed.legend()
display(fig_perturbed.canvas)

In [ ]:
manifold.resonance_area()

In [ ]:
areas = manifold.areas
areas

In [ ]:
areas[areas > 0].sum()